## Assessment of the impact of OCR on linguistic tasks

In particular, in this notebook we're looking at:
- Part-of-speech tagging accuracy (fine- and coarse-grained)
- Named entity recognition accuracy (matching type, matching type and IOB-tag)
    - Persons: f-score (by quality band)
    - Geopolitical entities: f-score (by quality band)
    - Dates: f-score (by quality band)
- Dependency parsing

In [1]:
import pathlib
import spacy
import pandas as pd
import numpy as np
import collections
import re
import ast
import matplotlib.pyplot as plt

import warnings; warnings.simplefilter('ignore')

import seaborn as sns
sns.set(style="white")
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})

In [2]:
nlp = spacy.load('en_core_web_lg')

### Get sample and apply Spacy

Obtain a sample of the data for which we have the same number of articles from each quality band. Apply Spacy to the OCR and the human-corrected texts, and save the new dataframe.

In [3]:
spacyfile = pathlib.Path("trove_subsample_aligned_spacy.pkl")
trovedf = pd.DataFrame()
sampledf_band1 = pd.DataFrame()
sampledf_band2 = pd.DataFrame()
sampledf_band3 = pd.DataFrame()
sampledf_band4 = pd.DataFrame()
sampledf = pd.DataFrame()
if not spacyfile.exists():
    trovedf = pd.read_pickle("trove_subsample_aligned.pkl")
    sampledf_band1 = trovedf[trovedf['quality_band'] == 1].sample(950)
    sampledf_band2 = trovedf[trovedf['quality_band'] == 2].sample(950)
    sampledf_band3 = trovedf[trovedf['quality_band'] == 3].sample(950)
    sampledf_band4 = trovedf[trovedf['quality_band'] == 4].sample(950)
    sampledf = pd.concat([sampledf_band1, sampledf_band2, sampledf_band3, sampledf_band4])
    sampledf['spacyOcr'] = sampledf['ocrText'].apply(nlp)
    sampledf['spacyHum'] = sampledf['corrected'].apply(nlp)
    sampledf.to_pickle('trove_subsample_aligned_spacy.pkl')
else:
    sampledf = pd.read_pickle('trove_subsample_aligned_spacy.pkl')

In [4]:
sampledf.head()

,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length,quality_band,use_corrected,alignment,processed,spacyOcr,spacyHum
11720,./trove_overproof/datasets/dataset1/rawTextAnd...,12968079,Article,1855,RUSSIA. A letter from St. Petersburg of tho 29...,RUSSIA. A letter from St. Petersburg of the 29...,RUSSIA. A letter from St. Petersburg of the 29...,0.920876,3397,1,0,"[(0, 7, 0, 7), (8, 9, 8, 9), (10, 16, 10, 16),...",yes,"(RUSSIA, ., A, letter, from, St., Petersburg, ...","(RUSSIA, ., A, letter, from, St., Petersburg, ..."
26626,./trove_overproof/datasets/dataset1/rawTextAnd...,14223127,Article,1900,AUSTRALIAN UNION BENEFIT SOCIETY. The annual g...,AUSTRALIAN UNION BENEFIT SOCIETY. The annual g...,AUSTRALIAN UNION BENEFIT SOCIETY. The annual g...,0.901502,2851,1,0,"[(0, 10, 0, 10), (11, 16, 11, 16), (17, 24, 17...",yes,"(AUSTRALIAN, UNION, BENEFIT, SOCIETY, ., The, ...","(AUSTRALIAN, UNION, BENEFIT, SOCIETY, ., The, ..."
14814,./trove_overproof/datasets/dataset1/rawTextAnd...,27972895,Article,1939,HOME NEWS; G.f'.O. Contract. Tim unexpected si...,HOME NEWS; G.P.O. Contract. The unexpected sig...,HOME NEWS; G.P.O. Contract. The unexpected sig...,0.916455,4716,1,0,"[(0, 4, 0, 4), (5, 10, 5, 10), (11, 18, 11, 17...",yes,"(HOME, NEWS, ;, G.f'.O., Contract, ., Tim, une...","(HOME, NEWS, ;, G.P.O., Contract, ., The, unex..."
18765,./trove_overproof/datasets/dataset1/rawTextAnd...,14817371,Article,1906,DIVORCE COURT. (Before Mr. Justice Simpson.) T...,DIVORCE COURT. (Before Mr. Justice Simpson.) T...,DIVORCE COURT. (Before Mr. Justice Simpson.) T...,0.921374,3021,1,0,"[(0, 7, 0, 7), (8, 14, 8, 14), (15, 22, 15, 22...",yes,"(DIVORCE, COURT, ., (, Before, Mr., Justice, S...","(DIVORCE, COURT, ., (, Before, Mr., Justice, S..."
20995,./trove_overproof/datasets/dataset1/rawTextAnd...,13324224,Article,1873,PORT MACRAT. DKFABTUBI. Octobers.-Hauaah Newto...,PORT MACKAY. DEPARTURE. Octobers 3.—Hannah New...,PORT MACKAY. DEPARTURE. Octobers 3.—Hannah New...,0.911892,1615,1,0,"[(0, 4, 0, 4), (5, 12, 5, 12), (41, 48, 43, 50...",yes,"(PORT, MACRAT, ., DKFABTUBI, ., Octobers.-Haua...","(PORT, MACKAY, ., DEPARTURE, ., Octobers, 3.—H..."


### Evaluate sentence splitting

In [5]:
def sentenceSplitting(spacyOcr, spacyHum, alignment):
    """
    We consider a sentence as being correctly split if
    both left and right boundaries enclose the exact same
    aligned tokens.
    """
    
    # dAligments is a dictionary with the following structure:
    # dAlignments = {spacyHum_startchar:spacyOcr_startchar}
    dAlignments = dict()
    for a in alignment:
        dAlignments[a[2]] = a[0]
    
    # humanSentences is a list of lists:
    # - the outer list is the list of sentences from the spacyHum column.
    # - each inner list is the list of words in each sentence, represented as the
    # word's start character.
    #
    # expectedOCRSentences is a list of lists:
    # - the outer list is the list of sentences from the human-corrected column as
    # well (i.e. the list of sentences we would expect if the OCR was identical to
    # the human-corrected text).
    # - each inner list is the list of words in each expected sentence, represented
    # as the start character of the OCR token that is aligned to the expected human-
    # corrected token. We therefore should expect humanSentences and expectedOCRSentences
    # to have the exact same shape.
    humanSentences = []
    expectedOCRSentences = []
    for sent in spacyHum.sents:
        newHumSentence = []
        expectedOCRSentence = []
        for a in dAlignments:
            if a in range(sent.start_char, sent.end_char):
                newHumSentence.append(a)
                expectedOCRSentence.append(dAlignments[a])
        humanSentences.append(newHumSentence)
        expectedOCRSentences.append(expectedOCRSentence)
    
    # ocrSentences is a list of lists:
    # - the outer list is the list of sentences from the spacyOcr column.
    # - each inner list is the list of words in each sentence in spacyOcr, represented
    # as the word's start character.
    # If the OCR text was identical to its human-correction, ocrSentences and
    # expectedOCRSentences will be identical.
    ocrSentences = []
    for sent in spacyOcr.sents:
        newOCRSentence = []
        for a in alignment:
            if a[0] in range(sent.start_char, sent.end_char):
                newOCRSentence.append(a[0])
        ocrSentences.append(newOCRSentence)
    
    # Calculate precision:
    # - true positive: each list of tokens that is exactly the same between the
    # expected OCR sentence and the true OCR sentence.
    # - false positive: each list of tokens that is found in the true OCR sentence
    # but not in the expected OCR sentence.
    tp_precision = 0
    fp_precision = 0
    for sent in ocrSentences:
        if sent:
            if sent in expectedOCRSentences:
                tp_precision += 1
            else:
                fp_precision += 1
    
    precision = 0
    if tp_precision + fp_precision > 0:
        precision = tp_precision / float(tp_precision + fp_precision)
    
    allsentences = 0
    
    # Calculate recall:
    # - true positive: each list of tokens that is exactly the same between the
    # expected OCR sentence and the true OCR sentence.
    # - false positive: each list of tokens that is found in the expected OCR sentence
    # but not in the true OCR sentence.
    tp_recall = 0
    fn_recall = 0
    for sent in expectedOCRSentences:
        if sent:
            allsentences += 1
            if sent in ocrSentences:
                tp_recall += 1
            else:
                fn_recall += 1
    
    recall = 0
    if tp_recall + fn_recall > 0:
        recall = tp_recall / float(tp_recall + fn_recall)
    
    # Calculate fscore:
    fscore = None
    if precision + recall > 0:
        fscore = (2.0 * precision * recall) / (precision + recall)
    
    # Calculate absolute accuracy:
    # Accuracy is calculated as, from all the expected OCR sentences, those
    # that are true OCR sentences.
    sentsegacc = None
    if allsentences > 0:
        sentsegacc = tp_recall / float(allsentences)
    
    return fscore, sentsegacc

In [6]:
# Test sentenceSplitting(...) function:
for i, row in sampledf.iloc[:1].iterrows():
    alignment = ast.literal_eval(row['alignment'])
    spacyOcr = row['spacyOcr']
    spacyHum = row['spacyHum']
    sentenceSplitting(spacyOcr, spacyHum, alignment)

### Evaluate named entity recognition

In [7]:
def evaluateNer(ocrText, humText, spacyOcrText, spacyHumText, absoluteIndexLinking, target):
    
    # dOcrTokenNer is a dictionary with the following structure:
    # dOcrTokenNer = {spacyOcrToken_startchar:(spacyOcrToken_entiob, spacyOcrToken_enttype, spacyOcrToken_text)}
    dOcrTokenNer = dict()
    for token in spacyOcrText:
        tokenIndicesNer = token.idx
        dOcrTokenNer[tokenIndicesNer] = (token.ent_iob_, token.ent_type_, token.text)
    
    # dHumTokenNer is a dictionary with the following structure:
    # dHumTokenNer = {spacyHumToken_startchar:(spacyHumToken_entiob, spacyHumToken_enttype, spacyHumToken_text)}
    # -> where entiob is the IOB tag (i.e. 'inside', 'outside', 'beginning'),
    # -> where enttype is the entity type (i.e. 'PER' for person, 'GPE' for geopolitical entity, 'DATE' for date).
    dHumTokenNer = dict()
    for token in spacyHumText:
        tokenIndicesNer = token.idx
        dHumTokenNer[tokenIndicesNer] = (token.ent_iob_, token.ent_type_, token.text)
    
    # Measures for accuracy:
    all_matched_words = 0 # overall
    correct_type = 0 # correct entity type
    correct_iob_type = 0 # correct entity type and iob tag
    
    # Per-type measures:
    person_tp = 0 # person true positives
    person_fp = 0 # person false positives
    person_fn = 0 # person false negatives
    gpe_tp = 0 # geo-political entities true positives
    gpe_fp = 0 # geo-political entities false positives
    gpe_fn = 0 # geo-political entities false negatives
    date_tp = 0 # date entities true positives
    date_fp = 0 # date entities false positives
    date_fn = 0 # date entities false negatives
    
    # Loop over the aligned tokens. Each matched_word is a tuple with the following strucure:
    # matched_word = (spacyOcrToken_startchar, spacyOcrToken_endchar, spacyHumToken_startchar, spacyHumToken_endchar)
    for matched_word in absoluteIndexLinking:
        if matched_word[0] in dOcrTokenNer and matched_word[2] in dHumTokenNer:
            labelOcr = dOcrTokenNer[matched_word[0]][1].strip() # entity type of ocr token
            labelHum = dHumTokenNer[matched_word[2]][1].strip() # entity type of aligned human-corrected token
            iobOcr = dOcrTokenNer[matched_word[0]][0].strip() # iob tag of ocr token
            iobHum = dHumTokenNer[matched_word[2]][0].strip() # iob tag of aligned human-corrected token
            
            # Target token:
            # -> "correct" if identical between OCR and human (i.e. alignment is a perfect string match), 
            # -> "incorrect" if OCR is different, 
            # -> "all" if we don't care
            target_condition = False
            if target == "correct":
                if dOcrTokenNer[matched_word[0]][2].lower() == dHumTokenNer[matched_word[2]][2].lower():
                    target_condition = True
            if target == "incorrect":
                if dOcrTokenNer[matched_word[0]][2].lower() != dHumTokenNer[matched_word[2]][2].lower():
                    target_condition = True
            if target == "all":
                target_condition = True
            
            # Different measures conditioned on the target:
            # -> correct_type = entity type of human token is the same as its corresponding ocr token
            # -> correct_iob_type = both entity type and IOB tag are the same
            # -> person_tp = entity type is 'person' both in ocr and hum token
            # -> gpe_tp = entity type is 'gpe' both in ocr and hum token
            # -> date_tp = entity type is 'date' both in ocr and hum token
            # -> person_fn = entity type is 'person' in ocr and something else in hum token
            # -> gpe_fn = entity type is 'gpe' in ocr and something else in hum token
            # -> date_fn = entity type is 'date' in ocr and something else in hum token
            # -> person_fp = entity type is 'person' in hum and something else ocr token
            # -> gpe_fp = entity type is 'gpe' in hum and something else ocr token
            # -> date_fp = entity type is 'date' in hum and something else ocr token
            if target_condition == True:
                all_matched_words += 1
                if labelOcr == labelHum:
                    correct_type += 1
                    if iobOcr == iobHum:
                        correct_iob_type += 1
                    if labelOcr == 'PERSON':
                        person_tp += 1
                    if labelOcr == 'GPE':
                        gpe_tp += 1
                    if labelOcr == 'DATE':
                        date_tp += 1
                if labelOcr != labelHum:
                    if labelOcr == 'PERSON':
                        person_fp += 1
                    if labelOcr == 'GPE':
                        gpe_fp += 1
                    if labelOcr == 'DATE':
                        date_fp += 1
                    if labelHum == 'PERSON':
                        person_fn += 1
                    if labelHum == 'GPE':
                        gpe_fn += 1
                    if labelHum == 'DATE':
                        date_fn += 1
    
    # Accuracy of entity type and IOB tag (not conditioned on the target):
    if all_matched_words == 0:
        accuracy_type = None
        accuracy_iob_type = None
    else:
        accuracy_type = correct_type/float(all_matched_words)
        accuracy_iob_type = correct_iob_type/float(all_matched_words)
    
    return accuracy_type, accuracy_iob_type, person_tp, person_fp, person_fn, gpe_tp, gpe_fp, gpe_fn, date_tp, date_fp, date_fn

In [8]:
# Test evaluateNer(...) function:
for i, row in sampledf.iloc[:1].iterrows():
    target = "correct" # 'correct', 'incorrect', 'all'
    ocrText = row['ocrText'].strip(" ")
    humanText = row['corrected'].strip(" ")
    spacyOcr = row['spacyOcr']
    spacyHum = row['spacyHum']
    alignment = ast.literal_eval(row['alignment'])
    evaluateNer(ocrText, humanText, spacyOcr, spacyHum, alignment, target)

### Evaluate dependency parsing

In [9]:
def evaluateDependencyParsing(noisyText, cleanText, spacyNoisyText, spacyCleanText, absoluteIndexLinking, dependency_length):
    dAlignedOcrHum = dict()
    dAlignedHumOcr = dict()
    ocrWords = []
    humWords = []
    
    # dAlignedOcrHum is a dictionary with the following structure:
    # dAlignedOcrHum = {spacyOcrToken_startchar : spacyHumToken_startchar}
    # dAlignedHumOcr = {spacyHumToken_startchar : spacyOcrToken_startchar}
    # 
    # ocrWords is the list of tokens from the spacyOcr that have been succesfully aligned,
    # represented as the start character of the token.
    # humWords is the list of tokens from the spacyHum that have been succesfully aligned,
    # represented as the start character of the token.
    for a in absoluteIndexLinking:
        ocrWords.append((a[0]))
        humWords.append((a[2]))
        dAlignedOcrHum[(a[0])] = a[2]
        dAlignedHumOcr[(a[2])] = a[0]
    
    # dNoisyTokenDeps is a dictionary with the following structure:
    # dNoisyTokenDeps = {spacyOcrToken_startChar : (spacyOcrToken_headStartChar, spacyOcrToken_dependencyType, spacyOcrToken_children, spacyOcrToken_deplength)}
    dNoisyTokenDeps = dict()
    for token in spacyNoisyText:
        noisy_dependency_span = []
        tokenIndex = (token.idx)
        tokenHeadIndex = (token.head.idx)
        if tokenIndex < tokenHeadIndex:
            noisy_dependency_span = noisyText[tokenIndex:tokenHeadIndex].strip().split()
        elif tokenIndex > tokenHeadIndex:
            noisy_dependency_span = noisyText[tokenHeadIndex:tokenIndex].strip().split()
        dNoisyTokenDeps[tokenIndex] = (tokenHeadIndex, token.dep_, [child for child in token.children], len(noisy_dependency_span))
    
    # dCleanTokenDeps is a dictionary with the following structure:
    # dCleanTokenDeps = {spacyHumToken_startChar : (spacyHumToken_headStartChar, spacyHumToken_dependencyType, spacyHumToken_children, spacyHumToken_deplength)}
    dCleanTokenDeps = dict()
    for token in spacyCleanText:
        clean_dependency_span = []
        tokenIndex = (token.idx)
        tokenHeadIndex = (token.head.idx)
        if tokenIndex < tokenHeadIndex:
            clean_dependency_span = cleanText[tokenIndex:tokenHeadIndex].strip().split()
        elif tokenIndex > tokenHeadIndex:
            clean_dependency_span = cleanText[tokenHeadIndex:tokenIndex].strip().split()
        dCleanTokenDeps[tokenIndex] = (tokenHeadIndex, token.dep_, [child for child in token.children], len(clean_dependency_span))
    
    # dNoisyDict is a dictionary with the following structure:
    # dNoisyDict = {(spacyOcrToken_startChar, spacyOcrToken_headStartChar) : (dependencyType, numberOfChildren, spacyOcrToken_deplength)}
    # -> where dependency type is the dependency type between the token and its head.
    # -> where numberOfChildren is the number of children of the token in question
    # -> where spacyOcrToken_deplength is the length of the dependency between the token and its head
    dNoisyDict = dict()
    for ocrWord1 in ocrWords:
        for ocrWord2 in ocrWords:
            if ocrWord1 in dNoisyTokenDeps:
                if ocrWord2 == dNoisyTokenDeps[ocrWord1][0]:
                    dNoisyDict[(ocrWord1, ocrWord2)] = (dNoisyTokenDeps[ocrWord1][1], len(dNoisyTokenDeps[ocrWord1][2]), dNoisyTokenDeps[ocrWord1][-1])
                    
    # dCleanDict is a dictionary with the following structure:
    # dCleanDict = {(spacyHumToken_startChar, spacyHumToken_headStartChar) : (dependencyType, numberOfChildren, spacyHumToken_deplength)}
    # -> where dependency type is the dependency type between the token and its head.
    # -> where numberOfChildren is the number of children of the token in question
    # -> where spacyHumToken_deplength is the length of the dependency between the token and its head
    dCleanDict = dict()
    for humWord1 in humWords:
        for humWord2 in humWords:
            if humWord1 in dCleanTokenDeps:
                if humWord2 == dCleanTokenDeps[humWord1][0]:
                    dCleanDict[(humWord1, humWord2)] = (dCleanTokenDeps[humWord1][1], len(dCleanTokenDeps[humWord1][2]), dCleanTokenDeps[humWord1][-1])
    
    # allOcrLabels is a counter of all dependencies for which we have an aligned token
    # in both sides of the dependency.
    # correctLabel is a counter of all the dependencies between aligned OCR and human-corrected
    # tokens that have the same entity type.
    allOcrLabels = 0
    correctLabel = 0
    for i in dNoisyDict:
        allOcrLabels += 1
        ocrTok = i[0]
        ocrHead = i[1]
        humTok = dAlignedOcrHum[i[0]]
        humHead = dAlignedOcrHum[i[1]]
        if (humTok, humHead) in dCleanDict:
            if dCleanDict[humTok, humHead][0] == dNoisyDict[i][0]:
                correctLabel += 1
        elif (humHead, humTok) in dCleanDict:
            if dCleanDict[humHead, humTok][0] == dNoisyDict[i][0]:
                correctLabel += 1
    if allOcrLabels == 0:
        precision = 0.0
    else:
        precision = correctLabel/float(allOcrLabels)
    
    # allHumLabels is a counter of all dependencies for which we have an aligned token
    # in both sides of the dependency.
    # correctLabel is a counter of all the dependencies between aligned OCR and human-corrected
    # tokens that have the same entity type.
    allHumLabels = 0
    correctLabel = 0    
    for i in dCleanDict:
        dep_condition = False
        # "all" for any distance between heads
        # "neighbour" for only neighbouring relations between heads
        # "close" for distance of three tokens between heads
        # "far" for distance of above five tokens between heads
        # "nsubj" for nsubj relationship
        # "compound" for compound relationship
        if dependency_length == "all":
            dep_condition = True
        elif dependency_length == "neighbour":
            if dCleanDict[i][-1] <= 1:
                dep_condition = True
        elif dependency_length == "close":
            if dCleanDict[i][-1] >= 3:
                dep_condition = True
        elif dependency_length == "far":
            if dCleanDict[i][-1] >= 5:
                dep_condition = True
        elif dependency_length == "nsubj":
            if dCleanDict[i][0] == "nsubj":
                dep_condition = True
        elif dependency_length == "compound":
            if dCleanDict[i][0] == "compound":
                dep_condition = True
        
        if dep_condition == True:
            allHumLabels += 1
            ocrTok = dAlignedHumOcr[i[0]]
            ocrHead = dAlignedHumOcr[i[1]]
            humTok = i[0]
            humHead =i[1]
            if (ocrTok, ocrHead) in dNoisyDict:
                if dNoisyDict[ocrTok, ocrHead][0] == dCleanDict[i][0]:
                    correctLabel += 1
            elif (ocrHead, ocrTok) in dNoisyDict:
                if dNoisyDict[ocrHead, ocrTok][0] == dCleanDict[i][0]:
                    correctLabel += 1
    if allOcrLabels == 0:
        recall = 0
    else:
        recall = correctLabel/float(allOcrLabels)
    
    if precision + recall == 0:
        fscore = None
    else:
        fscore = (2.0 * precision * recall) / (precision + recall)
    return fscore

In [10]:
# Test evaluateDependencyParsing(...) function:
for i, row in sampledf.iloc[:1].iterrows():
    target = "correct" # 'correct', 'incorrect', 'all'
    dependency_length = "neighbour" # 'all', 'neighbour', 'close', 'far', 'nsubj', 'compound'
    ocrText = row['ocrText'].strip(" ")
    humanText = row['corrected'].strip(" ")
    spacyOcr = row['spacyOcr']
    spacyHum = row['spacyHum']
    alignment = ast.literal_eval(row['alignment'])
    evaluateDependencyParsing(ocrText, humanText, spacyOcr, spacyHum, alignment, dependency_length)

In [11]:
# "correct" if the target token is identical between OCR and Hum text,
# "incorrect" if not, 
# "all" if we want all cases
target = "all"

# "all" for any distance between heads
# "neighbour" for only neighbouring relations between heads
# "close" for distance of three tokens between heads
# "far" for distance of above five tokens between heads
# "nsubj" for nsubj relationship
# "compound" for compound relationship
dependency_length = "all"

In [ ]:
sampledf['str_similarity'] = sampledf['str_similarity'].astype(float)
sampledf["sent_fscore"] = ""
sampledf["sent_acc"] = ""
sampledf["neriobtype_acc"] = ""
sampledf["nertype_acc"] = ""
sampledf["ner_person_acc"] = ""
sampledf["ner_per_tp"] = ""
sampledf["ner_per_fp"] = ""
sampledf["ner_per_fn"] = ""
sampledf["ner_gpe_tp"] = ""
sampledf["ner_gpe_fp"] = ""
sampledf["ner_gpe_fn"] = ""
sampledf["ner_date_tp"] = ""
sampledf["ner_date_fp"] = ""
sampledf["ner_date_fn"] = ""
sampledf["neriobtype_acc"] = ""
sampledf["dependency_parsing"] = ""
for index, row in sampledf.iterrows():
    if row['alignment'] != "":
        ocrText = row['ocrText'].strip(" ")
        humanText = row['corrected'].strip(" ")
        spacyOcr = row['spacyOcr']
        spacyHum = row['spacyHum']
        alignment = ast.literal_eval(row['alignment'])
        sentFscore, sentAcc = sentenceSplitting(spacyOcr, spacyHum, alignment)
        accEnttype, accEntiobtype, person_tp, person_fp, person_fn, gpe_tp, gpe_fp, gpe_fn, date_tp, date_fp, date_fn = evaluateNer(ocrText, humanText, spacyOcr, spacyHum, alignment, target)
        dep_parsing = evaluateDependencyParsing(ocrText, humanText, spacyOcr, spacyHum, alignment, dependency_length)
        sampledf.loc[index, "sent_fscore"] = sentFscore
        sampledf.loc[index, "sent_acc"] = sentAcc
        sampledf.loc[index, 'nertype_acc'] = accEnttype
        sampledf.loc[index, 'neriobtype_acc'] = accEntiobtype
        sampledf.loc[index, "ner_per_tp"] = person_tp
        sampledf.loc[index, "ner_per_fp"] = person_fp
        sampledf.loc[index, "ner_per_fn"] = person_fn
        sampledf.loc[index, "ner_gpe_tp"] = gpe_tp
        sampledf.loc[index, "ner_gpe_fp"] = gpe_fp
        sampledf.loc[index, "ner_gpe_fn"] = gpe_fn
        sampledf.loc[index, "ner_date_tp"] = date_tp
        sampledf.loc[index, "ner_date_fp"] = date_fp
        sampledf.loc[index, "ner_date_fn"] = date_fn
        sampledf.loc[index, 'dependency_parsing'] = dep_parsing

In [ ]:
sampledf.head()

### Plot sentence segmentation accuracy

In [ ]:
sent_filtered_df = sampledf[~sampledf['sent_acc'].isnull()]
sent_filtered_df = sent_filtered_df[sampledf['sent_acc'] != ""]

plt.figure(figsize=(10, 10))
plt.plot(sent_filtered_df['str_similarity'], sent_filtered_df['sent_acc'], 'ko', alpha=0.2)

plt.text(0.95, 0.8, 'Quality band 1', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.85, 0.8, 'Quality band 2', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.75, 0.8, 'Quality band 3', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.65, 0.8, 'Quality band 4', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')

plt.grid()
plt.xticks(size=24)
plt.yticks(size=24)
plt.xlabel("String similarity (Levenshtein)", size=28)
plt.ylabel("Sentence segmentation accuracy", size=28)
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.axvline(0.9, 0, 1, c='r', ls='--')
plt.axvline(0.8, 0, 1, c='r', ls='--')
plt.axvline(0.7, 0, 1, c='r', ls='--')
plt.savefig("figures/linguistic_analysis/sent_split.png", dpi=200)

### Plot named entity recognition accuracy

In [ ]:
ner_filtered_df = sampledf[~sampledf['neriobtype_acc'].isnull()]
ner_filtered_df = ner_filtered_df[sampledf['neriobtype_acc'] != ""]

plt.figure(figsize=(10, 10))
plt.plot(ner_filtered_df['str_similarity'], ner_filtered_df['neriobtype_acc'], 'ko', alpha=0.2)

plt.text(0.95, 0.05, 'Quality band 1', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.85, 0.05, 'Quality band 2', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.75, 0.05, 'Quality band 3', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.65, 0.05, 'Quality band 4', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')

plt.grid()
plt.xticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], size=24)
plt.yticks(size=24)
plt.xlabel("String similarity (Levenshtein)", size=28)
plt.ylabel("Named entity recognition accuracy", size=28)
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.axvline(0.9, 0, 1, c='r', ls='--')
plt.axvline(0.8, 0, 1, c='r', ls='--')
plt.axvline(0.7, 0, 1, c='r', ls='--')
plt.savefig("figures/linguistic_analysis/ner_acc.png", dpi=200)

### Plot named dependency parsing accuracy

In [ ]:
dep_filtered_df = sampledf[~sampledf['dependency_parsing'].isnull()]
dep_filtered_df = dep_filtered_df[sampledf['dependency_parsing'] != ""]

plt.figure(figsize=(10, 10))
plt.plot(dep_filtered_df['str_similarity'], dep_filtered_df['dependency_parsing'], 'ko', alpha=0.2)

plt.text(0.95, 0.8, 'Quality band 1', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.85, 0.8, 'Quality band 2', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.75, 0.8, 'Quality band 3', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')
plt.text(0.65, 0.8, 'Quality band 4', fontsize=16,
        rotation=90, rotation_mode='anchor', color='r')

plt.grid()
plt.xticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], size=24)
plt.yticks(size=24)
plt.xlabel("String similarity (Levenshtein)", size=28)
plt.ylabel("Dependency parsing accuracy", size=28)
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.axvline(0.9, 0, 1, c='r', ls='--')
plt.axvline(0.8, 0, 1, c='r', ls='--')
plt.axvline(0.7, 0, 1, c='r', ls='--')
plt.savefig("figures/linguistic_analysis/dep_parsing.png", dpi=200)

In [ ]:
sampledf_band1 = sampledf[sampledf['quality_band'] == 1]
sampledf_band2 = sampledf[sampledf['quality_band'] == 2]
sampledf_band3 = sampledf[sampledf['quality_band'] == 3]
sampledf_band4 = sampledf[sampledf['quality_band'] == 4]

In [ ]:
print(sampledf_band1['dependency_parsing'].mean())
print(sampledf_band2['dependency_parsing'].mean())
print(sampledf_band3['dependency_parsing'].mean())
print(sampledf_band4['dependency_parsing'].mean())

In [ ]:
dfbands = [sampledf_band1, sampledf_band2, sampledf_band3, sampledf_band4]
for enttype in ['per', 'gpe', 'date']:
    print(enttype)
    for band in reversed(range(len(dfbands))):
        per_tp = dfbands[band]['ner_' + enttype + '_tp'].sum(axis=0)
        per_fp = dfbands[band]['ner_' + enttype + '_fp'].sum(axis=0)
        per_fn = dfbands[band]['ner_' + enttype + '_fn'].sum(axis=0)

        precision = per_tp / (per_tp + per_fp)
        recall = per_tp / (per_tp + per_fn)
        fscore = (2 * precision * recall) / (precision + recall)
        print("\tBand", str(band + 1) + ":", fscore)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

labels = ['per', 'gpe', 'date']

bands_fscores = []
for band in range(len(dfbands)):
    entityfscores = []
    for enttype in labels:
        per_tp = dfbands[band]['ner_' + enttype + '_tp'].sum(axis=0)
        per_fp = dfbands[band]['ner_' + enttype + '_fp'].sum(axis=0)
        per_fn = dfbands[band]['ner_' + enttype + '_fn'].sum(axis=0)

        precision = per_tp / (per_tp + per_fp)
        recall = per_tp / (per_tp + per_fn)
        fscore = (2 * precision * recall) / (precision + recall)
        entityfscores.append(fscore)
    bands_fscores.append(entityfscores)

x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - 0.30, bands_fscores[0], width, label='Band1')
rects2 = ax.bar(x - 0.10, bands_fscores[1], width, label='Band2')
rects3 = ax.bar(x + 0.10, bands_fscores[2], width, label='Band3')
rects4 = ax.bar(x + 0.30, bands_fscores[3], width, label='Band4')

ax.set_ylabel('F-score', size=14)
ax.set_xticks(x)
ax.set_xticklabels(['PERSON', 'GPE', 'DATE'], size=14)
ax.legend()
plt.legend(loc='lower left')

fig.tight_layout()
plt.grid()
plt.savefig("figures/linguistic_analysis/ner_per_entity.png", dpi=200)